# 05. Baseline and Statistical Models

## 1. Objective
- Xây dựng mô hình chuẩn so sánh
- Áp dụng mô hình thống kê có thời vụ

## 2. Baseline Models
### 2.1 Naive Forecast
### 2.2 Seasonal Naive Forecast

## 3. Statistical Models
### 3.1 ARIMA
- Order selection
- ACF / PACF

### 3.2 SARIMA
- Seasonal order
- Daily & weekly seasonality

## 4. Model Training
- Huấn luyện trên tập train
- Kiểm tra residuals

## 5. Preliminary Results
- Dự báo ngắn hạn
- Nhận xét ban đầu

## 6. Conclusion
- Vai trò của statistical models